In [1]:
import os
import time
import re
from bs4 import BeautifulSoup
import pandas as pd
import requests
from urllib.parse import quote
import logging
import sys
import json
from apify_client import ApifyClient

In [2]:
# Load env
WEBHOOK_URL = os.environ["DISCORD_WEBHOOK_URL"]
APIFY_TOKEN = os.environ["APIFY_TOKEN"]

In [5]:
# Setup search parameters
city = 'taguig'
radius = '10' # in KM
product = 'arctis nova pro'
days_listed = 1
keywords = [
    'Karting',
    'Kart rim',
    'Kart frame',
    'Kart wheel hub'
]
url = f"https://www.facebook.com/marketplace/{quote(city)}/search?sortBy=best_match&query={quote(product)}&exact=false&radius_in_km={quote(radius)}"

In [6]:
# Initialize the ApifyClient with your API token
client = ApifyClient(APIFY_TOKEN)

# Prepare the Actor input
run_input = {
    "startUrls": [
        { "url": url },
    ],
    "resultsLimit": 10,
    "includeListingDetails": False,
}

# Run the Actor and wait for it to finish
run = client.actor("U5DUNxhH3qKt5PnCf").call(run_input=run_input)

# Fetch the Run details specifically for usage
run_details = client.run(run['id']).get()

# Extract key metrics
compute_units = run_details.get('computeUnits')
total_cost_usd = run_details.get('usageTotalUsd') # This is the "usage" in dollars
duration_secs = run_details.get('runTimeSecs')

print(f"Run completed in {duration_secs}s")
print(f"Compute Units used: {compute_units}")
print(f"Total Cost: ${total_cost_usd}")

# Fetch and print Actor results from the run's dataset (if there are any)
for item in client.dataset(run["defaultDatasetId"]).iterate_items():
    print(item)

[apify.facebook-marketplace-scraper runId:5rgR1gASGSXddiLH2] -> Status: RUNNING, Message: 
[apify.facebook-marketplace-scraper runId:5rgR1gASGSXddiLH2] -> 2026-02-07T07:35:13.027Z ACTOR: Pulling container image of build nGWgT3FdHxLv2NwNs from registry.
[apify.facebook-marketplace-scraper runId:5rgR1gASGSXddiLH2] -> 2026-02-07T07:35:13.029Z ACTOR: Creating container.
[apify.facebook-marketplace-scraper runId:5rgR1gASGSXddiLH2] -> 2026-02-07T07:35:13.101Z ACTOR: Starting container.
[apify.facebook-marketplace-scraper runId:5rgR1gASGSXddiLH2] -> 2026-02-07T07:35:13.103Z ACTOR: Running under "LIMITED_PERMISSIONS" permission level.
[apify.facebook-marketplace-scraper runId:5rgR1gASGSXddiLH2] -> 2026-02-07T07:35:13.884Z INFO  System info {"apifyVersion":"3.5.3","apifyClientVersion":"2.19.0","crawleeVersion":"3.15.2","osType":"Linux","nodeVersion":"v22.21.1"}
[apify.facebook-marketplace-scraper runId:5rgR1gASGSXddiLH2] -> 2026-02-07T07:35:13.989Z INFO  Results Limit 10
[apify.facebook-marketp

Run completed in Nones
Compute Units used: None
Total Cost: $0.062
{'facebookUrl': 'https://www.facebook.com/marketplace/taguig/search?sortBy=best_match&query=arctis%20nova%20pro&exact=false&radius_in_km=10', 'listingUrl': 'https://www.facebook.com/marketplace/item/884523624352756', 'id': '884523624352756', 'primary_listing_photo': {'__typename': 'CatalogMarketplaceEnhancementTransformedImage', 'id': '25369211199448116', 'photo_image_url': 'https://scontent-hou1-1.xx.fbcdn.net/v/t39.84726-6/621186921_25369211226114780_3695813042128941533_n.jpg?stp=c0.43.261.261a_dst-jpg_p261x260_tt6&_nc_cat=110&ccb=1-7&_nc_sid=92e707&_nc_ohc=CWzxbEx_inQQ7kNvwEikx21&_nc_oc=AdncjXa2u2tq0qvFlRch_HT_g2laQ-Jp2B0WjhMoD2qBP2lenpPoCPoD31BFlqW9BOQ&_nc_zt=14&_nc_ht=scontent-hou1-1.xx&_nc_gid=icW2M22nN3WxLfE6w0CU5g&oh=00_AfvkiFnppb9S6lZsEvDrzAhvR5PEyWvlQuHM7fhOL2niKA&oe=698CB0D9'}, 'listing_price': {'formatted_amount': 'PHP8,500', 'amount_with_offset_in_currency': '14481', 'amount': '8500.00'}, 'strikethrough_pri

In [14]:
limits_url = f"https://api.apify.com/v2/users/me/limits?token={APIFY_TOKEN}"

response = requests.get(limits_url)
if response.status_code == 200:
    stats = response.json()['data']
    remaining = stats['limits']['maxMonthlyUsageUsd'] - stats['current']['monthlyUsageUsd']
    print(f"Verified Balance: ${remaining:.2f}")

Verified Balance: $4.94


In [18]:
# Loaded data for sampling
sample_data = "./sample_data.jsonl"
with open(sample_data, "w") as file:
    for item in client.dataset(run["defaultDatasetId"]).iterate_items():
        json.dump(item, file)
        file.write('\n')